In [2]:
!pip install tensorflow tensorflow_hub librosa soundfile scikit-learn matplotlib


   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/332.0 MB 5.2 MB/s eta 0:01:04
   ---------------------------------------- 2.9/332.0 MB 6.9 MB/s eta 0:00:48
    --------------------------------------- 4.5/332.0 MB 7.3 MB/s eta 0:00:45
    --------------------------------------- 6.3/332.0 MB 7.6 MB/s eta 0:00:43
    --------------------------------------- 7.6/332.0 MB 7.8 MB/s eta 0:00:42
   - -------------------------------------- 9.2/332.0 MB 7.5 MB/s eta 0:00:44
   - -------------------------------------- 10.7/332.0 MB 7.5 MB/s eta 0:00:43
   - -------------------------------------- 14.2/332.0 MB 8.5 MB/s eta 0:00:38
   -- ------------------------------------- 17.8/332.0 MB 9.5 MB/s eta 0:00:33
   -- ------------------------------------- 21.0/332.0 MB 10.2 MB/s eta 0:00:31
   -- ------------------------------------- 24.6/332.0 MB 10.8 MB/s eta 0:00:29
   --- ------------------------------------ 26.2/332.0 MB 10.9 M

In [3]:
import os
import glob
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import librosa
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, callbacks

# -------------------------------------------------------
# CONFIG
# -------------------------------------------------------
DATA_DIR = "2C"                       # dataset folder
EMBED_CACHE = "embeddings_2C.npz"    # cache file
SAMPLE_RATE = 16000
EPOCHS = 30
BATCH_SIZE = 32
SEED = 42

# -------------------------------------------------------
# Load YAMNet (full version for training on laptop)
# -------------------------------------------------------
YAMNET_MODEL = "https://tfhub.dev/google/yamnet/1"
yamnet = hub.load(YAMNET_MODEL)   # full YAMNet TF model

# -------------------------------------------------------
# Helper: load wav & resample to 16 kHz mono
# -------------------------------------------------------
def load_audio(path, sr=SAMPLE_RATE):
    wav, orig_sr = librosa.load(path, sr=None, mono=True)
    if orig_sr != sr:
        wav = librosa.resample(wav, orig_sr, sr)
    return wav.astype(np.float32)

# -------------------------------------------------------
# Extract embeddings (or load cached)
# -------------------------------------------------------
def build_or_load_embeddings():
    if os.path.exists(EMBED_CACHE):
        print("Loading cached embeddings:", EMBED_CACHE)
        data = np.load(EMBED_CACHE, allow_pickle=True)
        return data["X"], data["y"], data["paths"]

    X_list, y_list, paths_list = [], [], []

    classes = sorted(os.listdir(DATA_DIR))
    print("Found classes:", classes)

    for cls in classes:
        cls_path = os.path.join(DATA_DIR, cls)
        if not os.path.isdir(cls_path):
            continue

        wav_files = glob.glob(os.path.join(cls_path, "*.wav"))
        print(f"Processing {cls}: {len(wav_files)} files")

        for p in wav_files:
            try:
                audio = load_audio(p)
                # Run YAMNet
                scores, embeddings, spect = yamnet(audio)
                emb_np = embeddings.numpy()

                # Mean pool over frames → (1024,)
                emb_mean = np.mean(emb_np, axis=0)

                X_list.append(emb_mean)
                y_list.append(cls)
                paths_list.append(p)
            except Exception as e:
                print("Error processing", p, e)

    X = np.vstack(X_list).astype(np.float32)
    y = np.array(y_list)
    paths = np.array(paths_list)

    np.savez(EMBED_CACHE, X=X, y=y, paths=paths)
    print("Saved embedding cache:", EMBED_CACHE)

    return X, y, paths

# -------------------------------------------------------
# MLP classifier model
# -------------------------------------------------------
def build_classifier(input_dim, num_classes):
    inp = layers.Input(shape=(input_dim,))
    x = layers.Dense(512, activation='relu')(inp)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    out = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inp, out)

# -------------------------------------------------------
# Main training pipeline
# -------------------------------------------------------
def main():
    print("Extracting or loading embeddings...")
    X, y, paths = build_or_load_embeddings()
    print("Embedding shape:", X.shape)

    # Encode labels
    le = LabelEncoder()
    y_enc = le.fit_transform(y)

    # Split: 70 / 15 / 15
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y_enc, test_size=0.30, stratify=y_enc, random_state=SEED
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=SEED
    )

    print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

    # Build model
    model = build_classifier(1024, len(le.classes_))
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    model.summary()

    # Callbacks
    es = callbacks.EarlyStopping(patience=5, restore_best_weights=True)
    ck = callbacks.ModelCheckpoint("fire_classifier.h5", save_best_only=True)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=[es, ck],
        verbose=2
    )

    # Evaluate on test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print("\nTest Accuracy:", test_acc)

    # Save label mapping
    with open("label_map.json", "w") as f:
        json.dump({"classes": le.classes_.tolist()}, f)

    print("\nSaved model: fire_classifier.h5")
    print("Saved label map: label_map.json")

if __name__ == "__main__":
    main()


Extracting or loading embeddings...
Found classes: ['fire', 'no_fire']
Processing fire: 1432 files
Processing no_fire: 1438 files
Saved embedding cache: embeddings_2C.npz
Embedding shape: (2870, 1024)
Train: (2009, 1024) Val: (430, 1024) Test: (431, 1024)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 590,722 (2.25 MB)

 Trainable params: 590,722 (2.25 MB)

...

63/63 - 0s - 7ms/step - accuracy: 0.9617 - loss: 0.1026 - val_accuracy: 0.9698 - val_loss: 0.0747
Epoch 3/30
63/63 - 0s - 6ms/step - accuracy: 0.9726 - loss: 0.0745 - val_accuracy: 0.9674 - val_loss: 0.0879
Epoch 4/30


63/63 - 0s - 7ms/step - accuracy: 0.9746 - loss: 0.0625 - val_accuracy: 0.9628 - val_loss: 0.0609
Epoch 5/30


63/63 - 0s - 7ms/step - accuracy: 0.9746 - loss: 0.0617 - val_accuracy: 0.9721 - val_loss: 0.0528
Epoch 6/30
63/63 - 0s - 6ms/step - accuracy: 0.9816 - loss: 0.0474 - val_accuracy: 0.9721 - val_loss: 0.0557
Epoch 7/30
63/63 - 0s - 6ms/step - accuracy: 0.9836 - loss: 0.0623 - val_accuracy: 0.9698 - val_loss: 0.0564
Epoch 8/30
63/63 - 0s - 6ms/step - accuracy: 0.9831 - loss: 0.0431 - val_accuracy: 0.9767 - val_loss: 0.0565
Epoch 9/30
63/63 - 0s - 6ms/step - accuracy: 0.9816 - loss: 0.0405 - val_accuracy: 0.9698 - val_loss: 0.0637
Epoch 10/30


63/63 - 0s - 7ms/step - accuracy: 0.9856 - loss: 0.0360 - val_accuracy: 0.9884 - val_loss: 0.0470
Epoch 11/30


63/63 - 0s - 7ms/step - accuracy: 0.9890 - loss: 0.0296 - val_accuracy: 0.9837 - val_loss: 0.0424
Epoch 12/30
63/63 - 0s - 6ms/step - accuracy: 0.9861 - loss: 0.0332 - val_accuracy: 0.9837 - val_loss: 0.0573
Epoch 13/30
63/63 - 0s - 6ms/step - accuracy: 0.9905 - loss: 0.0325 - val_accuracy: 0.9767 - val_loss: 0.0588
Epoch 14/30
63/63 - 0s - 6ms/step - accuracy: 0.9890 - loss: 0.0280 - val_accuracy: 0.9791 - val_loss: 0.0620
Epoch 15/30
63/63 - 0s - 6ms/step - accuracy: 0.9890 - loss: 0.0245 - val_accuracy: 0.9814 - val_loss: 0.0600
Epoch 16/30
63/63 - 0s - 6ms/step - accuracy: 0.9930 - loss: 0.0219 - val_accuracy: 0.9814 - val_loss: 0.0502

Test Accuracy: 0.9860788583755493

Saved model: fire_classifier.h5
Saved label map: label_map.json


In [4]:
import tensorflow as tf

model = tf.keras.models.load_model("fire_classifier.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("fire_classifier.tflite", "wb") as f:
    f.write(tflite_model)

print("✔ Saved fire_classifier.tflite")
          

INFO:tensorflow:Assets written to: C:\Users\MRUTYU~1\AppData\Local\Temp\tmpiweg0sl8\assets


INFO:tensorflow:Assets written to: C:\Users\MRUTYU~1\AppData\Local\Temp\tmpiweg0sl8\assets


Saved artifact at 'C:\Users\MRUTYU~1\AppData\Local\Temp\tmpiweg0sl8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1024), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  2584590803088: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2584590805200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2584590805008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2584590806352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2584590804816: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2584590801168: TensorSpec(shape=(), dtype=tf.resource, name=None)
✔ Saved fire_classifier.tflite
